# Automatic Sandpit Refinement for D-Flow FM

This notebook provides an automated workflow for refining unstructured grids around sandpit areas in D-Flow FM models. The refinement process uses Casulli refinement to gradually transition from coarse background resolution to fine target resolution.

## Workflow Overview

1. **Configuration**: Set file paths and refinement parameters
2. **Load Grid and Polygons**: Load the grid and create/load sandpit polygons
3. **Plan Refinement**: Analyze grid resolution and generate refinement zones
4. **Execute Refinement**: Apply Casulli refinement to the grid
5. **Monitor Quality** (Optional): Analyze grid quality metrics

## Requirements

- `dfm_tools`
- `meshkernel`
- `numpy`
- `matplotlib`
- `shapely`

In [1]:
# Step 0: Configuration and Setup

import numpy as np
import os
import dfm_tools as dfmt
import matplotlib.pyplot as plt

# Import utility functions
from src.polygon_utils import InteractivePolygonDrawer, load_pol_file, save_pol_file, generate_refinement_polygons, expand_polygon_outward
from src.refinement_utils import compute_refinement_steps, apply_casulli_refinement, print_refinement_summary
from src.visualization_utils import plot_grid, set_interactive_plots, set_static_plots
from src.monitoring_utils import analyze_grid_quality, plot_grid_quality

# Base path is the workspace root in Codespace
main_path = '/workspaces/orelse-sandpit-automation'  # or just use '.'
nc_file = 'dcsm_0_125nm_2ref_bathygr7_RGFGRID_net.nc'
nc_path = os.path.join(main_path, 'data', 'input', nc_file)

# Output settings
output_dir = os.path.join(main_path, 'data', 'output')
os.makedirs(output_dir, exist_ok=True)

# User configuration
use_existing_pol_file = True
existing_pol_file = os.path.join(main_path, 'data', 'input', 'sandpits.pol')

# Refinement parameters
target_resolution = 30  # Target resolution in meters for the finest refinement level
buffer_around_sandpit = 250  # Buffer around sandpit polygons in meters
N = 7  # Number of transition cells

print("✅ Configuration loaded")
print(f"   Target: {target_resolution}m | Buffer: {buffer_around_sandpit}m | Transitions: {N}")

✅ Configuration loaded
   Target: 30m | Buffer: 250m | Transitions: 7


## Step 1: Load Grid and Define Sandpit Polygons

This step loads the D-Flow FM grid and either:
- Loads existing sandpit polygons from a .pol file
- Opens an interactive drawing tool to create new polygons

The polygons define the areas where refinement will be applied.

In [4]:
# Step 1: Load Grid and Create/Load Polygons

# Load grid data
print("📊 Loading grid...")
ugrid = dfmt.open_partitioned_dataset(nc_path)
ugrid_original = dfmt.open_partitioned_dataset(nc_path)  # Keep original for later comparison

# Convert to meshkernel objects for consistent plotting and refinement
mk_object = ugrid.grid.meshkernel
mk_backup = ugrid_original.grid.meshkernel  # Keep backup

print(f"✅ Grid loaded: {ugrid.sizes['nmesh2d_node']} nodes, {ugrid.sizes['nmesh2d_face']} faces")

# Handle polygon input/creation
if use_existing_pol_file:
    # Load existing polygon file
    if os.path.exists(existing_pol_file):
        polygons = load_pol_file(existing_pol_file)
        print(f"✅ Loaded {len(polygons)} sandpit polygons from file")
    else:
        print(f"❌ File {existing_pol_file} not found, switching to interactive mode")
        use_existing_pol_file = False

if not use_existing_pol_file:
    # Interactive polygon creation
    set_interactive_plots()
    print("🖱️  Opening interactive polygon drawing tool...")
    print("   Instructions: RIGHT CLICK → add vertex | ENTER → finish polygon | Close window when done")
    
    drawer = InteractivePolygonDrawer(ugrid, nc_path)
    polygons = drawer.draw_polygons()
    
    if polygons:
        output_file = os.path.join(output_dir, 'sandpits.pol')
        save_pol_file(polygons, output_file)
        print(f"✅ Saved {len(polygons)} polygons to {output_file}")

📊 Loading grid...
>> xu.open_dataset() with 1 partition(s): 1 [nodomainvar] : 0.56 sec
>> xu.open_dataset() with 1 partition(s): 1 [nodomainvar] : 0.19 sec
✅ Grid loaded: 843868 nodes, 835646 faces
✅ Loaded 4 sandpit polygons from file


## Step 2: Plan Refinement Strategy

This step:
1. Analyzes the current grid resolution within the sandpit polygons
2. Calculates the number of refinement steps needed
3. Generates refinement zones with automatic overlap detection and merging
4. Visualizes the refinement plan

The refinement zones are created from coarse (outer) to fine (inner) resolution.

In [5]:
# Step 2: Plan Refinement Strategy

# Analyze grid resolution and compute refinement steps
print("🔍 Analyzing grid resolution...")
refinement_params = compute_refinement_steps(ugrid, target_resolution, polygons)

# Generate refinement polygons with overlap merging
print("📐 Generating refinement zones...")
(all_refinement_polygons, all_original_polygons, 
 buffer_polygons, expansions) = generate_refinement_polygons(
    polygons, refinement_params, buffer_around_sandpit, N)

# Store original buffer polygons for visualization
original_buffer_polygons = []
for i, polygon in enumerate(polygons):
    polygon_array = np.array(polygon)
    center_lat = np.mean(polygon_array[:, 1])
    expanded_polygon = expand_polygon_outward(polygon, buffer_around_sandpit, center_lat)
    original_buffer_polygons.append(expanded_polygon)

# Visualize refinement plan
print("📈 Visualizing refinement plan...")
set_static_plots()
plot_grid(mk_object, polygons, all_refinement_polygons, all_original_polygons,
          buffer_polygons, refinement_params['envelope_sizes_m'], refinement_params['n_steps'],
          original_buffer_polygons=original_buffer_polygons,
          title='Refinement Plan: Sandpit Polygons and Merged Refinement Zones')
plt.show()

🔍 Analyzing grid resolution...
Grid analysis: Current 460m → Target 30m in 4 steps
📐 Generating refinement zones...
Generating 4 refinement levels: 460m → 30m
  Step 1: merged 2 overlapping polygons
  Level 1: 4 → 3 polygons @ 29m
  Step 2: merged 2 overlapping polygons
  Level 2: 4 → 3 polygons @ 58m
  Level 3: 4 → 4 polygons @ 115m
  Level 4: 4 → 4 polygons @ 230m
Buffer polygons: 4 → 4 @ 250m
📈 Visualizing refinement plan...
📈 Static plots enabled


## Step 3: Execute Grid Refinement

This step applies Casulli refinement to the meshkernel object using the generated refinement zones. The refinement is applied from outside to inside (coarse to fine) to ensure smooth transitions.

After refinement, the refined grid is visualized showing the final mesh structure.

In [ ]:
# Step 3: Execute Grid Refinement

# Apply Casulli refinement
print("⚙️  Executing refinement...")
apply_casulli_refinement(mk_object, all_refinement_polygons)

# Visualize refined grid
print("📊 Visualizing refined grid...")
set_static_plots()
plot_grid(mk_object, polygons, all_refinement_polygons, all_original_polygons,
          buffer_polygons, refinement_params['envelope_sizes_m'], refinement_params['n_steps'],
          title='Refined Grid: Final Result with Casulli Refinement')

# Print refinement summary
print_refinement_summary(polygons, all_refinement_polygons, 
                        refinement_params['envelope_sizes_m'], 
                        refinement_params['n_steps'], buffer_polygons)

# Save refined grid if needed
# ugrid_refined = dfmt.meshkernel_to_UgridDataset(mk_object, crs='EPSG:4326')
# output_nc = os.path.join(output_dir, 'refined_grid.nc')
# ugrid_refined.to_netcdf(output_nc)
# print(f"💾 Refined grid saved to: {output_nc}")

## Step 4: Monitor Grid Quality (Optional)

This optional step analyzes the quality of the refined grid by examining:
- **Resolution**: Face areas converted to characteristic lengths
- **Smoothness**: Ratio of adjacent cell sizes (target < 1.4)
- **Orthogonality**: Deviation from 90° angles (target < 0.01)

Note: This analysis can be computationally intensive for large grids.

In [ ]:
# Step 4: (Optional) Monitor Grid Quality

# Uncomment the following lines to perform grid quality analysis
# Note: This can be computationally intensive for large grids

print("🔬 Analyzing grid quality metrics...")
quality_data = analyze_grid_quality(mk_object, ugrid_original, all_refinement_polygons, polygons)

print("📊 Creating quality visualization...")
set_static_plots()
plot_grid_quality(quality_data, all_refinement_polygons, target_resolution)

## Additional Operations (Optional)

Here are some additional operations you might want to perform after refinement:

In [ ]:
# Optional: Export refined grid to NetCDF
save_refined_grid = False  # Set to True if you want to save

if save_refined_grid:
    print("💾 Saving refined grid...")
    ugrid_refined = dfmt.meshkernel_to_UgridDataset(mk_object, crs='EPSG:4326')
    output_nc = os.path.join(output_dir, 'refined_grid.nc')
    ugrid_refined.to_netcdf(output_nc)
    print(f"✅ Refined grid saved to: {output_nc}")

# Optional: Compare grid statistics
print("\n📊 Grid comparison:")
original_nodes = ugrid_original.sizes['mesh2d_nNodes']
original_faces = ugrid_original.sizes['mesh2d_nFaces']

# Get refined grid stats
refined_mesh = mk_object.mesh2d_get()
refined_nodes = len(refined_mesh.node_x)
refined_faces = len(refined_mesh.face_x)

print(f"   Original: {original_nodes:,} nodes, {original_faces:,} faces")
print(f"   Refined:  {refined_nodes:,} nodes, {refined_faces:,} faces")
print(f"   Increase: {refined_nodes/original_nodes:.1f}x nodes, {refined_faces/original_faces:.1f}x faces")

## Workflow Complete! 🎉

The automatic sandpit refinement workflow has been completed. Your D-Flow FM grid has been refined around the sandpit areas with:

- **Progressive refinement** from coarse background to fine target resolution
- **Automatic overlap handling** for multiple sandpits
- **Quality monitoring** to ensure good mesh properties
- **Visualization** of the refinement process and results

The refined grid is now ready for use in your D-Flow FM simulations!

### Next Steps:
1. **Export the grid** (uncomment the save section above if needed)
2. **Transfer to your D-Flow FM model setup**
3. **Update bathymetry** if needed for the refined areas
4. **Adjust model parameters** for the new resolution
5. **Run test simulations** to validate the refined grid